# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7efff834dee0>
├── 'a' --> tensor([[-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604]])
└── 'x' --> <FastTreeValue 0x7efff834df10>
    └── 'c' --> tensor([[ 0.5156,  1.1729, -0.7799, -2.0367],
                        [ 1.2157, -1.0141, -1.2673,  0.5998],
                        [-0.5272,  0.1644, -0.4966, -1.2734]])

In [4]:
t.a

tensor([[-0.7408, -1.3222,  0.8525],
        [-0.4305, -1.2665, -1.6604]])

In [5]:
%timeit t.a

64.2 ns ± 0.0147 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7efff834dee0>
├── 'a' --> tensor([[-0.8265,  0.0878,  0.3411],
│                   [-0.1591,  0.3817,  1.5782]])
└── 'x' --> <FastTreeValue 0x7efff834df10>
    └── 'c' --> tensor([[ 0.5156,  1.1729, -0.7799, -2.0367],
                        [ 1.2157, -1.0141, -1.2673,  0.5998],
                        [-0.5272,  0.1644, -0.4966, -1.2734]])

In [7]:
%timeit t.a = new_value

63.5 ns ± 0.0267 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604]]),
    x: Batch(
           c: tensor([[ 0.5156,  1.1729, -0.7799, -2.0367],
                      [ 1.2157, -1.0141, -1.2673,  0.5998],
                      [-0.5272,  0.1644, -0.4966, -1.2734]]),
       ),
)

In [10]:
b.a

tensor([[-0.7408, -1.3222,  0.8525],
        [-0.4305, -1.2665, -1.6604]])

In [11]:
%timeit b.a

59.3 ns ± 0.0452 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4754, -1.8471,  2.7430],
               [ 1.1620, -0.5706,  0.9849]]),
    x: Batch(
           c: tensor([[ 0.5156,  1.1729, -0.7799, -2.0367],
                      [ 1.2157, -1.0141, -1.2673,  0.5998],
                      [-0.5272,  0.1644, -0.4966, -1.2734]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.129 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

877 ns ± 14.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 25.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 380 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7eff52b9daf0>
├── 'a' --> tensor([[[-0.7408, -1.3222,  0.8525],
│                    [-0.4305, -1.2665, -1.6604]],
│           
│                   [[-0.7408, -1.3222,  0.8525],
│                    [-0.4305, -1.2665, -1.6604]],
│           
│                   [[-0.7408, -1.3222,  0.8525],
│                    [-0.4305, -1.2665, -1.6604]],
│           
│                   [[-0.7408, -1.3222,  0.8525],
│                    [-0.4305, -1.2665, -1.6604]],
│           
│                   [[-0.7408, -1.3222,  0.8525],
│                    [-0.4305, -1.2665, -1.6604]],
│           
│                   [[-0.7408, -1.3222,  0.8525],
│                    [-0.4305, -1.2665, -1.6604]],
│           
│                   [[-0.7408, -1.3222,  0.8525],
│                    [-0.4305, -1.2665, -1.6604]],
│           
│                   [[-0.7408, -1.3222,  0.8525],
│                    [-0.4305, -1.2665, -1.6604]]])
└── 'x' --> <FastTreeValue 0x7efff834dfd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 76.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7efff82e1b50>
├── 'a' --> tensor([[-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604],
│                   [-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604],
│                   [-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604],
│                   [-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604],
│                   [-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604],
│                   [-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604],
│                   [-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604],
│                   [-0.7408, -1.3222,  0.8525],
│                   [-0.4305, -1.2665, -1.6604]])
└── 'x' --> <FastTreeValue 0x7eff4bfe2be0>
    └── 'c' --> tensor([[ 0.5156,  1.1729, -0.7799, -2.0367],
                        [ 1.2157, -1.0141, -1.2673,  0.5998],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 47.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.9 µs ± 59.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.7408, -1.3222,  0.8525],
                [-0.4305, -1.2665, -1.6604]],
       
               [[-0.7408, -1.3222,  0.8525],
                [-0.4305, -1.2665, -1.6604]],
       
               [[-0.7408, -1.3222,  0.8525],
                [-0.4305, -1.2665, -1.6604]],
       
               [[-0.7408, -1.3222,  0.8525],
                [-0.4305, -1.2665, -1.6604]],
       
               [[-0.7408, -1.3222,  0.8525],
                [-0.4305, -1.2665, -1.6604]],
       
               [[-0.7408, -1.3222,  0.8525],
                [-0.4305, -1.2665, -1.6604]],
       
               [[-0.7408, -1.3222,  0.8525],
                [-0.4305, -1.2665, -1.6604]],
       
               [[-0.7408, -1.3222,  0.8525],
                [-0.4305, -1.2665, -1.6604]]]),
    x: Batch(
           c: tensor([[[ 0.5156,  1.1729, -0.7799, -2.0367],
                       [ 1.2157, -1.0141, -1.2673,  0.5998],
                       [-0.5272,  0.1644, -0.4966, -1.2734]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604],
               [-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604],
               [-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604],
               [-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604],
               [-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604],
               [-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604],
               [-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604],
               [-0.7408, -1.3222,  0.8525],
               [-0.4305, -1.2665, -1.6604]]),
    x: Batch(
           c: tensor([[ 0.5156,  1.1729, -0.7799, -2.0367],
                      [ 1.2157, -1.0141, -1.2673,  0.5998],
                      [-0.5272,  0.1644, -0.4966, -1.2734],
                      [ 0.5156,  1.1729, -0.7799, -2.0367],
                      [ 1.2157, -

In [28]:
%timeit Batch.cat(batches)

141 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 646 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
